# SQL:

Yuanfeng Li 
2021/07/22

### Review support materials:
- [sql 复习汇总](https://zhuanlan.zhihu.com/p/43810990)
- [Leetcode Database 全部练习题及题解](https://zhuanlan.zhihu.com/p/265247944)

In [ ]:
# 175, combine two tables:
# My Answer is wrong, because I did not understand 'regardless if there is an address for each of those people'
# Correct Answer: 

# Write your MySQL query statement below
SELECT P.FirstName, P.LastName, A.City, A.State
FROM Person as P LEFT JOIN Address as A
ON P.PersonId = A.PersonId;

In [ ]:
# 176:

# Leetcode answer:
SELECT
 (SELECT DISTINCT Salary
  FROM Employee
  ORDER BY Salary DESC
  LIMIT 1 OFFSET 1) 
AS SecondHighestSalary;

# my version 1 year ago
# Write your MySQL query statement below
SELECT IFNULL(
    (SELECT DISTINCT Salary 
    FROM Employee 
    ORDER BY Salary DESC
    LIMIT 1 OFFSET 1), NULL
) AS SecondHighestSalary;


# my current version is not right, 
# I forget that limit 1 offset 1
SELECT DISTINCT Salary AS SecondHighestSalary
  FROM Employee
  ORDER BY Salary DESC
  LIMIT 1 OFFSET 1
# this one will not return null, it will return an empty 


In [ ]:
# 177
# tutorial url: https://www.mysqltutorial.org/mysql-limit.aspx

CREATE FUNCTION getNthHighestSalary(N INT) RETURNS INT
BEGIN
    DECLARE temp INTEGER;
    SET temp = N-1;
  RETURN (
      # Write your MySQL query statement below.
      SELECT IFNULL(
          (SELECT DISTINCT Salary 
           FROM Employee 
           ORDER BY Salary DESC 
           LIMIT temp,1), 
          NULL)
  );
END

In [ ]:
# 178 Medium:
SELECT Score,
DENSE_RANK() OVER(ORDER BY Score DESC) AS 'Rank'
FROM Scores

# 非常好的学西链接：https://zhuanlan.zhihu.com/p/135119865
# 讲述了 rank(), Dense_Rank(), Row_number() 的不同！


In [ ]:
# 180： M
# Lead() and Lag()
SELECT DISTINCT Num AS ConsecutiveNums FROM(
    SELECT Num,
    LEAD(Num, 1) OVER(ORDER BY id) AS lead1,
    LEAD(Num, 2) OVER(ORDER BY id) AS lead2
    FROM Logs
) AS lg
WHERE Num = lead1
AND Num = lead2


# or Leetcode solution:
SELECT *
FROM
Logs l1,
Logs l2,
Logs l3
WHERE
l1.Id = l2.Id - 1
AND l2.Id = l3.Id - 1
AND l1.Num = l2.Num
AND l2.Num = l3.Num


In [ ]:
# 181.

# 自己写的：
/* Write your T-SQL query statement below * /
SELECT A.Name AS Employee
FROM Employee as A
INNER JOIN Employee as B
ON A.ManagerId = B.Id
WHERE A.Salary > B.Salary


In [ ]:
# 182.

# 之前自己写过，但是忘了：
SELECT Email
FROM Person
GROUP BY Email
HAVING COUNT(Email) > 1

# LeetCode Answer:
SELECT Email
FROM(
    SELECT Email, COUNT(id) AS total
    FROM Person
    GROUP BY Email
) AS tmp
WHERE total > 1


# 自己想的方法
# Write your MySQL query statement below
WITH tmp AS
(SELECT Email, COUNT(id) as total
 FROM Person
 GROUP BY Email)
SELECT Email
FROM tmp
WHERE total > 1


In [ ]:
# leetcode answer!
DELETE p1.* FROM Person p1,
Person p2
WHERE
p1.Email = p2.Email AND p1.Id > p2.Id

# another acceptable answer: (even failed)
SELECT MIN(Id) as Id, Email
FROM Person
GROUP BY Email;


# Write your MySQL query statement below


SELECT Name AS Customers
FROM Customers AS C
WHERE C.Id NOT IN(
    SELECT C.Id
    FROM Orders AS O
    JOIN Customers AS C
    ON C.Id=O.CustomerId
)

'''
92.75%
This Question is Quiet Tricky!

hint, what if they have the same names?

'''

# Leetcode 答案，简单直观！
SELECT Name AS 'Customers'
FROM Customers
WHERE Id NOT IN(
    SELECT CustomerId FROM Orders
)


In [ ]:
# 184:
# 自己想的， 太繁琐，太麻烦
WITH tmp AS(
    SELECT DepartmentId, MAX(Salary) AS S
    FROM Employee
    GROUP BY DepartmentId
)
SELECT D.Name as Department, E.Name As Employee, E.Salary
FROM Department As D
JOIN Employee AS E
ON D.Id = E.DepartmentId
JOIN tmp AS T ON
E.DepartmentId = T.DepartmentId
WHERE E.Salary = T.S

# 自己看了一遍答案写出来的：
SELECT D.Name AS Department, E.Name as Employee, E.Salary As Salary
FROM Employee As E JOIN
Department AS D ON
E.DepartmentId = D.Id
WHERE(E.DepartmentId, E.Salary) IN(
    SELECT DepartmentId, MAX(Salary)
    FROM Employee
    GROUP BY DepartmentId
)

# leetcode答案，only work for MySQL

SELECT
    Department.name AS 'Department',
    Employee.name AS 'Employee',
    Salary
FROM
    Employee
        JOIN
    Department 
        ON 
    Employee.DepartmentId = Department.Id
WHERE
    (Employee.DepartmentId , Salary) IN
    (   SELECT
            DepartmentId, MAX(Salary)
        FROM
            Employee
        GROUP BY DepartmentId
    );


In [ ]:
# 196 delete duplicate Emails
## leetcode Answer:
DELETE FROM Person
WHERE Id NOT IN
(SELECT MinId FROM
 (SELECT Email, MIN(Id) AS MinId FROM Person
  GROUP BY Email) AS tmp)


# my refined version
DELETE FROM Person
WHERE Id NOT IN(
    SELECT MINId
    FROM(
        SELECT MIN(Id) AS MINId, Email
        FROM Person
        GROUP BY Email
    ) AS tmp
)

# (Mine with error!)
DELETE FROM Person
WHERE (Email, Id) NOT IN
(SELECT Email, MIN(Id) AS Id 
FROM Person
GROUP BY Email)
## error!
# You can't specify target table 'Person' for update in FROM clause
# see this link: https://stackoverflow.com/questions/45494/mysql-error-1093-cant-specify-target-table-for-update-in-from-clause

## the following one did not work, since it was asked you to delete instead
## of select!, the show in leetcode called the original table.
SELECT Id, Email
FROM Person
WHERE(Id, Email)
IN
(SELECT MIN(Id) AS Id, Email
 FROM Person
 GROUP BY Email)


## Create 
# create table in sqlite3 to test my thoughts

# first create a db, so you can access what you have built
sqlite3 alibaba.db # create if not exists.

CREATE Table Person (
    Id INTEGER PRIMARY KEY,
    Email TEXT
); 

CREATE Table IF NOT EXISTS Person(
    Id INTEGER PRIMARY KEY,
    Email TEXT
);

# how to drop table
DROP TABLE IF EXISTS Person;

# see if there is still tables
.table

# INSERT 
INSERT INTO Person (Id, Email) 
VALUES 
    (1, 'john@example.com'),
    (2, 'bob@example.com'),
    (3, 'john@example.com')

.table

SELECT * FROM Person 

# better view:
.mode column
.header on 

# (https://www.oreilly.com/library/view/using-sqlite/9781449394592/re63.html)

# now I can finally join it with itself!
SELECT * 
FROM Person p1, Person p2
WHERE p1.Email = p2.Email;

# what did I say man! this is the double for loop!! Now i
'''
Id          Email             Id          Email
----------  ----------------  ----------  ----------------
1           john@example.com  1           john@example.com
1           john@example.com  3           john@example.com
2           bob@example.com   2           bob@example.com
3           john@example.com  1           john@example.com
3           john@example.com  3           john@example.com

'''

# New Selection!
SELECT *
FROM Person p1, Person p2
WHERE p1.Email = p2.Email AND p1.Id > p2.Id;

'''
Id          Email             Id          Email
----------  ----------------  ----------  ----------------
3           john@example.com  1           john@example.com


'''


# 分类：
- Join的应用：
    - 175 ez
    - 181 ez

- 多个select:
    - 176 ez
    
- Create Function:
    - 177 med
    - 
- Aggregation: having, group by etc
    - 182

- SubQueries:
    - Not In: 
        - 183

- 窗口函数：
    - 复习学习链接：https://zhuanlan.zhihu.com/p/250429998
    - 178 ez (DENSE_RANK())
    - 180 md (LEAD(,))

- 增删改查：
    - 196 (delete, specify problem)


In [ ]:
# 1241. Number of Comments per post

